In [1]:
import os

In [2]:
api_key = os.getenv("GEMINI_API_KEY")

In [12]:
from typing import Any, Dict

from google.adk.agents import Agent, LlmAgent, SequentialAgent, ParallelAgent
from google.adk.apps.app import App, EventsCompactionConfig
from google.adk.models.google_llm import Gemini
from google.adk.sessions import DatabaseSessionService
from google.adk.sessions import InMemorySessionService
from google.adk.runners import Runner, InMemoryRunner
from google.adk.tools import AgentTool, FunctionTool, google_search
from google.adk.tools.tool_context import ToolContext
from google.adk.code_executors import BuiltInCodeExecutor
from google.genai import types

print("Sessions and context engineering (3a) and MAS (1b) and Tools (2a) ADK imports")

Sessions and context engineering (3a) and MAS (1b) and Tools (2a) ADK imports


In [4]:
APP_NAME = "Wellness Suite"
USER_ID = "Hector"
SESSION = "default"
# USER_PROFILE = {
#     "name": USER_ID,
#     "age": None,
#     "weight": None,
#     "height": None,
#     "goals": [],
#     "injuries": [],
#     "dietary_restrictions": []
# }

MODEL_NAME = "gemini-2.5-flash-lite"

In [5]:
retry_config = types.HttpRetryOptions(
    attempts=3,
    exp_base=7, # Delay multiplier
    initial_delay=1, # initial delay before first retry (seconds)
    http_status_codes=[429, 500, 503, 504] # Retry on these HTTP errors
)

In [6]:
# Set up Session Management (stored in RAM)
session_service = InMemorySessionService()

In [7]:
# Helper functions for running a session
async def run_session(
    runner_instance: Runner,
    user_queries: list[str] | str = None,
    session_name: str = "default",
):
    print(f"\n ### Session: {session_name}")

    # Get app name from the Runner
    app_name = runner_instance.app_name

    # Attempt to create a new session or retrieve an existing one
    try:
        session = await session_service.create_session(
            app_name=app_name, user_id=USER_ID, session_id=session_name
        )
    except:
        session = await session_service.get_session(
            app_name=app_name, user_id=USER_ID, session_id=session_name
        )

    # Process queries if provided
    if user_queries:
        # Convert single query to list for uniform processing
        if type(user_queries) == str:
            user_queries = [user_queries]

        # Process each query in the list sequentially
        for query in user_queries:
            print(f"\nUser > {query}")

            # Convert the query string to the ADK content format
            query = types.Content(role="user", parts=[types.Part(text=query)])

            # Stream the agent's response asynchronously
            async for event in runner_instance.run_async(
                user_id=USER_ID, session_id=session.id, new_message=query
            ):
                # Check if the event contains valid content
                if event.content and event.content.parts:
                    # Filter out empty or "None" responses before printing
                    if (
                        event.content.parts[0].text != "None"
                        and event.content.parts[0].text
                    ):
                        print(f"{MODEL_NAME} > ", event.content.parts[0].text)
    else:
        print("No queries!")

In [8]:
# Specialist Agents

# Fitness Trainer Agent: focuses on fitness
fitness_trainer_agent = Agent(
    name="FitnessTrainer",
    model=Gemini(
        model=MODEL_NAME,
        retry_options=retry_config
    ),
    instruction="""You are an expert fitness trainer with deep knowledge of human physiology.
    Please provide a detailed fitness training plan for the user that is customized for the following:
    1. Their current level of fitness
    2. To help them achieve their fitness goals
    3. Takes into consideration their injury history
    4. Sets realistic time frames for developing lifting technique and expertise
    """,
    tools=[],
    output_key="fitness_plan"
)

# Nutritionist Agent: focuses on nutrition
nutritionist_agent = Agent(
    name="Nutritionist",
    model=Gemini(
        model=MODEL_NAME,
        retry_options=retry_config
    ),
    instruction="""You are an expert nutritionist with deep knowledge of diet and human metabolism.
    Please provide a detailed nutrition plan for the user that is customized for the following:
    1. Their dietary restrictions
    2. To help them achieve their weight and fitness goals
    3. Suggest meal plans that align with their preferences
    4. Ensure that the nutrition plan meets the macronutrient and micronutrient needs to achieve their goals
    """,
    tools=[],
    output_key="nutrition_plan"
)

In [9]:
# Aggregator Agent

aggregator_agent = Agent(
    name="WellnessAggregator",
    model=Gemini(
        model=MODEL_NAME,
        retry_options=retry_config
    ),
    instruction="""Combine the fitness plan and nutrition plan into a single wellness plan tailored for the user.
    Focus on the synergy between the fitness and nutrition aspects to maximize the user's health and wellness outcomes.

    **Fitness Plan:**
    {fitness_plan}

    **Nutrition Plan:**
    {nutrition_plan}

    Your final wellness plan should seamlessly integrate both components and provide clear, actionable recommendations.
    """,
    output_key="wellness_plan"
)

In [10]:
# Parallel Agent to run both specialists simultaneously
parallel_wellness_team = ParallelAgent(
    name="ParallelWellnessTeam",
    sub_agents=[fitness_trainer_agent, nutritionist_agent],
)

In [11]:
# Sequential Agent to run the parallel team first, then run the aggregator
root_agent = SequentialAgent(
    name="WellnessSystem",
    sub_agents=[parallel_wellness_team, aggregator_agent],
)

In [13]:
runner = InMemoryRunner(agent=root_agent)
response = await runner.run_debug(
    """I am a 30 year old male who weighs 170 lbs and is 5 foot 6 inches tall.
    I have previous lifting and cardio experience but have fallen out of shape and would like to lose 20 lbs while gaining muscle.
    Please create a detailed wellness plan for me."""
)


 ### Created new session: debug_session_id

User > I am a 30 year old male who weighs 170 lbs and is 5 foot 6 inches tall.
    I have previous lifting and cardio experience but have fallen out of shape and would like to lose 20 lbs while gaining muscle.
    Please create a detailed wellness plan for me.
Nutritionist > It's great that you're looking to get back into shape and focus on both fat loss and muscle gain! This is a very achievable goal with a structured approach to nutrition and exercise. As your nutritionist, I'll craft a plan to help you reach your targets.

First, let's establish some baseline information:

*   **Age:** 30 years old
*   **Sex:** Male
*   **Weight:** 170 lbs
*   **Height:** 5'6" (66 inches)
*   **Goal:** Lose 20 lbs (target weight of 150 lbs) and gain muscle.

**Understanding Your Needs:**

To lose fat, you need to be in a calorie deficit. To gain muscle, you need adequate protein intake and resistance training. The key is to create a deficit that is not so

In [14]:
runner_session = Runner(agent=root_agent, app_name=APP_NAME, session_service=session_service)

In [15]:
await run_session(
    runner_session,
    [
        """I am a 30 year old male who weighs 170 lbs and is 5 foot 6 inches tall.
    I have previous lifting and cardio experience but have fallen out of shape and would like to lose 20 lbs while gaining muscle.
    Please create a detailed wellness plan for me.
        """
    ],
    "stateful-agentic-session",
)


 ### Session: stateful-agentic-session

User > I am a 30 year old male who weighs 170 lbs and is 5 foot 6 inches tall.
    I have previous lifting and cardio experience but have fallen out of shape and would like to lose 20 lbs while gaining muscle.
    Please create a detailed wellness plan for me.
        
gemini-2.5-flash-lite >  It's great you're looking to get back into shape and achieve your goals! Losing 20 lbs while gaining muscle is a common and achievable objective with the right approach. This plan will focus on a sustainable nutrition strategy and provide guidance on how to structure your training.

Let's break down your needs:

**1. Calculating Your Baseline Needs:**

First, we need to estimate your Basal Metabolic Rate (BMR) and Total Daily Energy Expenditure (TDEE).

*   **BMR (Harris-Benedict Equation - Revised):**
    *   For men: BMR = 88.362 + (13.397 \* weight in kg) + (4.799 \* height in cm) - (5.677 \* age in years)
    *   Weight in kg: 170 lbs / 2.205 = 77.1 kg